# Use text for stock regression price prediction

- https://www.kaggle.com/code/lseiyjg/use-news-to-predict-stock-markets

In [1]:
import numpy as np
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2", use_fast=False)

Some weights of the model checkpoint at vinai/phobert-base-v2 were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens

In [2]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [14]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import mysql.connector

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    host='127.0.0.1',
    port=13306,
    user='root',
    password='root',
    database='pyml'
)

# Read the table data using pandas
query = "SELECT title, content, date FROM crawl_data where domain = 'https://vneconomy.vn/kinh-te-the-gioi.htm'"
df = pd.read_sql(query, connection)

# df = pd.read_csv('../../data/crawl_data_202306210731.csv')

print(df.head())

     id                                     domain  \
0  2389  https://vneconomy.vn/kinh-te-the-gioi.htm   
1  2390  https://vneconomy.vn/kinh-te-the-gioi.htm   
2  2391  https://vneconomy.vn/kinh-te-the-gioi.htm   
3  2392  https://vneconomy.vn/kinh-te-the-gioi.htm   
4  2393  https://vneconomy.vn/kinh-te-the-gioi.htm   

                                                 url  \
0  /tin-hieu-u-am-ve-kinh-te-toan-cau-tu-cac-nha-...   
1  /argentina-tang-lai-suat-len-gan-70-de-chong-l...   
2  /50-trieu-can-ho-khong-nguoi-o-qua-bom-no-cham...   
3  /bua-tiec-trai-phieu-bat-dong-san-trung-quoc-d...   
4  /bi-kip-thanh-cong-cua-ceo-cong-ty-duoc-menh-d...   

                                               title  \
0  Tín hiệu u ám về kinh tế toàn cầu từ các nhà m...   
1  Argentina tăng lãi suất lên gần 70% để chống l...   
2  50 triệu căn hộ không người ở: “Quả bom nổ chậ...   
3  “Bữa tiệc” trái phiếu bất động sản Trung Quốc ...   
4  Bí kíp thành công của CEO công ty được mệnh da...   

 

In [15]:
df['date_only'] = pd.to_datetime(df['date'], format='%Y-%m-%d %H:%M:%S').dt.strftime('%Y-%m-%d')
print(df['date_only'].head())

0    2022-08-15
1    2022-08-15
2    2022-08-15
3    2022-08-15
4    2022-08-15
Name: date_only, dtype: object


In [16]:
from vnstock import *
from sklearn.model_selection import train_test_split

df_his = stock_historical_data('TPB', '2022-01-01', '2023-07-28', "1D", 'stock')
df_his['date'] = pd.to_datetime(df_his['time']).dt.strftime('%Y-%m-%d')
print(df_his.head())

         time   open   high    low  close    volume ticker        date
0  2022-01-04  26870  28000  26740  27680  11034900    TPB  2022-01-04
1  2022-01-05  27520  27680  27130  27190   3828100    TPB  2022-01-05
2  2022-01-06  26740  27840  26740  27190   5190400    TPB  2022-01-06
3  2022-01-07  27090  27450  26550  26550   4150700    TPB  2022-01-07
4  2022-01-10  26550  27160  26190  26250   3892800    TPB  2022-01-10


In [17]:
print(df.info())
print(df_his.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1498 entries, 0 to 1497
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1498 non-null   int64  
 1   domain      1498 non-null   object 
 2   url         1498 non-null   object 
 3   title       1498 non-null   object 
 4   content     1498 non-null   object 
 5   date        1498 non-null   object 
 6   tags        0 non-null      float64
 7   created_at  1498 non-null   object 
 8   date_only   1498 non-null   object 
dtypes: float64(1), int64(1), object(7)
memory usage: 105.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 391 entries, 0 to 390
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   time    391 non-null    object
 1   open    391 non-null    int64 
 2   high    391 non-null    int64 
 3   low     391 non-null    int64 
 4   close   391 non-null    int64 
 5   volume  391 n

In [18]:
dfMerge = pd.merge(df, df_his, left_on=['date_only'], right_on=['date'], how='inner')
dfSumarize = dfMerge[['title', 'close', 'date_y']]
print(dfSumarize.head())

                                               title  close      date_y
0  Tín hiệu u ám về kinh tế toàn cầu từ các nhà m...  18740  2022-08-15
1  Argentina tăng lãi suất lên gần 70% để chống l...  18740  2022-08-15
2  50 triệu căn hộ không người ở: “Quả bom nổ chậ...  18740  2022-08-15
3  “Bữa tiệc” trái phiếu bất động sản Trung Quốc ...  18740  2022-08-15
4  Bí kíp thành công của CEO công ty được mệnh da...  18740  2022-08-15


In [19]:
# Sorting the DataFrame by the 'date_column' in ascending order

df_sorted = dfSumarize.sort_values(by='date_y', ascending=True)
print(df_sorted.head())

                                               title  close      date_y
0  Tín hiệu u ám về kinh tế toàn cầu từ các nhà m...  18740  2022-08-15
9  Vàng sụt giá sáng đầu tuần, chuyên gia cảnh bá...  18740  2022-08-15
8  “Đế chế” dầu lửa vùng Vịnh lãi gần 50 tỷ USD t...  18740  2022-08-15
7  Hungary được Nga bơm nhiều khí đốt hơn, Đức lo...  18740  2022-08-15
6  Năm công ty nhà nước Trung Quốc đồng loạt hủy ...  18740  2022-08-15


In [32]:
from nis import cat
from exceptiongroup import catch
from torch import embedding
from torch.nn.functional import pad
from underthesea import classify, word_tokenize, sent_tokenize
from pickle import load

max_len = 258

def extract_hidden_state(outputs, type='view'):
    # 2 way for get hidden state: outputs[0], outputs['last_hidden_state']
    with torch.no_grad():
        if type == 'view':
            hiddenSize = outputs[0].size()[0]
            # flatten dimension 1 and 2, and hold batch dimension 0
            embeddings = outputs[0].view(hiddenSize, -1).numpy()
        elif type == 'mean':
            hiddenSize = outputs[0].size()[0]
            embeddings = outputs[0].mean(axis=1).numpy()

    return embeddings

def vectorize_mean(sentences, type_extract='mean'):
    encodedText = tokenizer.batch_encode_plus(
        sentences,
        add_special_tokens=True,
        truncation=True, 
        padding='max_length',
        return_attention_mask=True,
        return_token_type_ids=False,
        max_length=max_len, return_tensors='pt'
        )
    encodedTextInputIds = encodedText['input_ids']
    embeddedTitle = phobert(torch.tensor(encodedTextInputIds))
    embedding = extract_hidden_state(embeddedTitle, type_extract)
    return embedding

def vectorize_mean_for(sentences, type_extract='mean'):
    embeddingArr = []
    for sentence in sentences:
        try:
            encodedText = tokenizer.encode_plus(
                sentence, padding=True, truncation=True, 
                add_special_tokens=True,
                return_attention_mask=True,
                max_length=300, return_tensors='pt')
        except Exception as e:
            print('e', e, sentence)
            return None
        
        encodedTextInputIds = encodedText['input_ids']
        embeddedTitle = phobert(torch.tensor(encodedTextInputIds))
        embedding = extract_hidden_state(embeddedTitle, type_extract)
        embeddingArr.append(embedding.ravel())
    return embeddingArr

def trim_words(sentence, num_word=200):
    # Split the sentence into words
    words = sentence.split()
    # Trim the sentence to 5 words
    trimmed_sentence = " ".join(words[:num_word])
    return trimmed_sentence


test = [
    'Lạm phát khiến Hàn Quốc giảm ngân sách lần đầu',
]

# t = vectorize_mean(test)
t = vectorize_mean_for(test)
print(t)



[array([-2.01829448e-02,  8.64872262e-02, -8.73286575e-02,  1.88893065e-01,
       -1.23140261e-01, -1.68129236e-01, -2.92146027e-01, -3.38564813e-01,
        2.12325752e-02,  1.26371041e-01,  8.35454166e-02,  5.97798750e-02,
        1.54909700e-01,  1.58262793e-02,  5.03614135e-02, -1.22325093e-01,
        2.16585770e-02,  2.43656784e-01, -6.98359534e-02,  2.20651329e-01,
       -8.83912891e-02,  7.27590695e-02, -1.29440024e-01, -3.01606767e-02,
       -3.38772833e-01,  3.08912486e-01,  1.22906407e-02,  3.96917872e-02,
       -2.48780832e-01, -1.77071065e-01, -2.27202073e-01,  8.64247903e-02,
        2.12022476e-02,  3.39687802e-02,  2.98975985e-02,  5.58604777e-01,
       -1.83685452e-01,  1.48079723e-01,  1.00964420e-01,  5.66273391e-01,
        1.44922987e-01,  1.14517875e-01,  1.71799272e-01,  1.07087240e-01,
        2.94297278e-01, -7.76214525e-02,  2.59682566e-01,  1.10911489e-01,
       -2.63571054e-01,  9.06947851e-02,  4.62090448e-02, -1.26314268e-01,
       -3.54175031e-01, 

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_3844/3106091539.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = phobert(torch.tensor(encodedTextInputIds))


In [33]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

x_train_raw = df_sorted['title'].values.tolist()

x_train = vectorize_mean_for(x_train_raw, type_extract='mean')

print(np.array(x_train).shape)

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_3844/3106091539.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = phobert(torch.tensor(encodedTextInputIds))


(1323, 768)


In [52]:

y_train_raw = df_sorted['close'][:10]
y_train = y_train_raw[:10].ravel()
x_train_t = x_train[:10]
print(x_train_t.shape, y_train)
# Step 2: Create and fit the SVR model
modelSvg = SVR(kernel='linear')
modelSvg.fit(x_train_t, y_train)  # ravel y_train to convert to 1D array



(10, 768) [18740 18740 18740 18740 18740 18740 18740 18740 18740 18740]


SVR(kernel='linear')

In [44]:
test = [
    'Lạm phát khiến Hàn Quốc giảm ngân sách lần đầu',
]

# t = vectorize_mean(test)
x_test_encoded = vectorize_mean_for(test)
print(len(x_test_encoded[0]), x_test_encoded)
result_pred = modelSvg.predict(x_test_encoded)
print(result_pred)

768 [array([-2.01829448e-02,  8.64872262e-02, -8.73286575e-02,  1.88893065e-01,
       -1.23140261e-01, -1.68129236e-01, -2.92146027e-01, -3.38564813e-01,
        2.12325752e-02,  1.26371041e-01,  8.35454166e-02,  5.97798750e-02,
        1.54909700e-01,  1.58262793e-02,  5.03614135e-02, -1.22325093e-01,
        2.16585770e-02,  2.43656784e-01, -6.98359534e-02,  2.20651329e-01,
       -8.83912891e-02,  7.27590695e-02, -1.29440024e-01, -3.01606767e-02,
       -3.38772833e-01,  3.08912486e-01,  1.22906407e-02,  3.96917872e-02,
       -2.48780832e-01, -1.77071065e-01, -2.27202073e-01,  8.64247903e-02,
        2.12022476e-02,  3.39687802e-02,  2.98975985e-02,  5.58604777e-01,
       -1.83685452e-01,  1.48079723e-01,  1.00964420e-01,  5.66273391e-01,
        1.44922987e-01,  1.14517875e-01,  1.71799272e-01,  1.07087240e-01,
        2.94297278e-01, -7.76214525e-02,  2.59682566e-01,  1.10911489e-01,
       -2.63571054e-01,  9.06947851e-02,  4.62090448e-02, -1.26314268e-01,
       -3.54175031e-

/var/folders/g6/37kt02914kx36yzcbbqfyck00000gn/T/ipykernel_3844/3106091539.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  embeddedTitle = phobert(torch.tensor(encodedTextInputIds))


(10, 15, 768)

In [47]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda, Embedding, LSTM, SimpleRNN, GRU
from keras.preprocessing.text import Tokenizer

x_train = np.array(x_train)
input_dim = x_train.shape[1]
nb_classes = 1
# Here's a Deep Dumb MLP (DDMLP)
modelDL = Sequential()
modelDL.add(Dense(256, input_dim=input_dim))
modelDL.add(Activation('relu'))
modelDL.add(Dropout(0.4))
modelDL.add(Dense(128))
modelDL.add(Activation('relu'))
modelDL.add(Dropout(0.4))
modelDL.add(Dense(nb_classes))
modelDL.add(Activation('softmax'))
modelDL.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(modelDL.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               196864    
                                                                 
 activation_6 (Activation)   (None, 256)               0         
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 activation_7 (Activation)   (None, 128)               0         
                                                                 
 dropout_5 (Dropout)         (None, 128)               0         
                                                                 
 dense_8 (Dense)             (None, 1)                

In [48]:
modelDL.fit(x_train, y_train)

42/42 [==============================] - 0s 2ms/step - loss: 0.0000e+00 - accuracy: 0.0000e+00
